<a href="https://colab.research.google.com/github/larasacodes/ML_Project_LaraAmusan/blob/main/Wine_Quality_Predictor_LaraAmusan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>